In [1]:
import numpy as np
from scipy import sparse as sp
from tqdm import trange

from src.hyperparameters_optimizer import HyperparametersOptimizer
from src.recommender_model import RecommenderModel
from src.utils import train_model, write_submission, open_dataset

In [2]:
class P3(RecommenderModel):
    def __init__(self):
        super(P3, self).__init__()
        self.alpha: float = 0
        self.beta: float = 0

    def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix, urm_val: sp.csr_matrix, progress_bar: bool = True, alpha: float = 1., beta: float = 0., batch_size: int = 4096) -> None:
        self.urm = urm
        urm_t = urm.T
        self.alpha = alpha
        self.beta = beta
        num_users, num_items = self.urm.shape

        deg_users = sp.diags(np.power(self.urm.sum(axis=1).A.ravel(), -1))
        deg_items = sp.diags(np.power(urm_t.sum(axis=1).A.ravel(), -1))

        p_ui = (deg_users @ self.urm).power(self.alpha)
        p_iu = (deg_items @ urm_t).power(self.alpha)

        popularity_scaling = np.power(urm_t.sum(axis=1).A, -beta)
        similarity = (p_iu @ p_ui).multiply(popularity_scaling)

        self.urm_pred = np.zeros((num_users, num_items), dtype=np.float32)

        iterator = trange(0, num_users, batch_size, desc="Users") if progress_bar else range(0, num_users, batch_size)

        for start_user_idx in iterator:
            num_users_batch = min(batch_size, num_users - start_user_idx)
            end_user_idx = start_user_idx + num_users_batch

            batch_diag = sp.diags(np.ones(num_users_batch), shape=(num_users_batch, num_users_batch))

            batch_pred = batch_diag @ p_ui[start_user_idx:end_user_idx, :] @ similarity
            self.urm_pred[start_user_idx:end_user_idx, :] = batch_pred.toarray()


In [3]:
optimizer = HyperparametersOptimizer({
	'alpha': np.arange(.9, 1.01, .1),
	'beta': np.arange(0, .41, .1),
}, P3)
_, best_parameters = optimizer.optimize()

100%|██████████| 10/10 [20:39<00:00, 123.91s/it, Best MAP@10: 0.0467 with ['alpha: 1.00e+00', 'beta: 2.00e-01']]


In [4]:
p3_submission = train_model(P3(), test_size=0, **best_parameters)
write_submission(p3_submission, "p3_submission.csv")

Users: 100%|██████████| 9/9 [02:55<00:00, 19.45s/it]


AttributeError: 'tuple' object has no attribute 'recommend'